## Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

# Подгружаем библеотеку для чтения субтитров
import pysrt
# Подгружаем библеотеку для получения списка специальных символов
import string
# Подгружаем библеотеки для выделения слов из текста
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Подгружаем библиотеку для преобразования слов в их исходное значение (к корню)
from nltk.stem import WordNetLemmatizer

# Подгружаем библиотеки для получения списка файлов из папки
from os import listdir
from os.path import isfile, join

## Функции используемые в проекте

In [2]:
# 1
# Функция для загрузки данных, хранящих информацию об уровнях английского
def func_load(file):
    
    # загрузим данные
    try:
        data_levels = pd.read_excel(file)
        
    except:
        print('Проблема с данными, попробуйте снова.')
    
    return data_levels

# 2
# Функция для формирования общего массива слов из фремов с уровнями английского
def united_words(*args):
    
    united = []
    
    # запустим обработку фреймов с учетом, того что их может быть много
    for arg in args:        
        for column in arg.columns:
            # пройдемся циклом по всем элементам из столбцов до появления пропусков
            for word in (np.array(arg[arg[column].isna() == False][column])):
                united.append(word)

    return united

# 3
# Функция для удаления лишних символов
def remove_symbol(text, symbols_to_delete):
    for symbol in symbols_to_delete:
        text = text.replace(str(symbol), '')
    return text

# 4
# Функция для разделения текста по словам с удалением стоп-слов
def transform_text(text):
    
    # разделяем текст по словам (токенам)
    word_tokens = word_tokenize(text)
    # приводим слова к изначальной форме
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    
    return lemmas

# 5
# Функция для чтения субтитров и преобразования их в удобный для обработки вид
def subs_transform(file):
    
    # чтение субтитров из файла
    subs = pysrt.open(file, encoding='iso-8859-1')

    # объединим субтитры в один единый текст
    subs_union = "".join([str(sub) for sub in subs])

    # уберём все лишние символы, такие как цифры и специальные символы вместе со знаком перевода строки
    subs_union = remove_symbol(subs_union, DIGITS + PUNCTUATIONS)

    # переведём всё в нижний регистр
    subs_union = subs_union.lower()
    
    # преобразуем текст субтитров в список слов
    subs_union = transform_text(subs_union)
    
    return subs_union

# 6
# Функция для масштабирования частоты появления слов
def freq_word_scal(subs, freq_word_subs):
        '''
        Данная функция необходима, для того избавится от проблемы больших и маленьких текстов.
        Имеется ввиду следующее, если фильм длинной около 3 часов, то количество слов в субтитрах 
        будет существенно больше чем в фильме длинной час. Для этого, будем находить долю частоты
        появления конкретного слова от общего числа слов в тексте. 
        '''
        # найдём общее количество слов в субтитрах
        count_words = len(subs)
        
        # запустим цикл по словарю частот слов, получая доли от общего числа слов в субтитрах
        for key in freq_word_subs.keys():
            try:
                freq_word_subs[key] = freq_word_subs[key] / count_words
            except:
                freq_word_subs[key] = 0
            
        return freq_word_subs
    
# 7
# Функция для подсчета частоты повторения слов
def frequency_word(list_word_from_text, list_word_by_levels):
    
    # объявим словарь, который будет собирать частоты появления слов
    freq_words = {}
    
    # запускаем цикл по словам из списка уникальных значений, на основе которых, 
    # выстраиваются уровни владения языком (уровневый список)
    for level_word in list_word_by_levels:
        # для удобства, вначале будем присваивать ноль для базового значения частот слов
        freq_words[level_word] = 0
        # запускаем цикл по словам из списка текста
        for text_word in list_word_from_text:
            # подсчёт частот производится на основе равенства слов из уровнего списка со словами из текста
            if(level_word == text_word):
                freq_words[level_word] += 1
                
    # произведём масштабирование частот слов
    freq_words = freq_word_scal(list_word_from_text, freq_words)
                
    return freq_words

# 8
# Функция для получения списка файлов в заданной папке
def func_list_files(folder_path):
    
    # получим список файлов
    list_files = [folder_path + f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    
    return list_files

# 9 
# Функция, осуществляющая разметку текстов из заданной папки, c получением общего фрейма данных
def text_markup(folder_path, combined_unique_words):
    
    # получим список файлов для разметки
    list_files = func_list_files(folder_path)
    
    # объявим матрицу частот слов, переменную для хранения субтитров,
    # массив, содержащий текущий набор частот слов и результирующий фрейм данных
    matrix_values = [] 
    subs = ''
    freq_words = []
    result = pd.DataFrame()
    
    # запустим обработку всех текстовых файлов в заданном каталоге
    for file in list_files:
        
        # обработка текущего файла субтитров
        subs = subs_transform(file)
        # получим массив значений частот слов
        freq_words = [val for val in frequency_word(subs, combined_unique_words).values()]
        
        # добавим получившуюся вектор-строку в матрицу значений
        matrix_values.append(freq_words)
        
    # сформируем обобщённый фрейм данных, который будет содержать частоты слов для каждого фильма 
    result = pd.DataFrame(data=matrix_values, columns=combined_unique_words)
    
    return result

In [3]:
# перечень используемых в проекте констант

# Цифры
DIGITS = ''.join([str(digit) for digit in range(10)])

# Знаки препинания и символ перевода строки
PUNCTUATIONS = string.punctuation + '\n'


## Загрузка данных

In [4]:
# загрузим данные по уровням для английского языка 
english_levels = func_load('British_levels.xlsx')
display(english_levels.head(5))

# загрузим данные по уровням для американского языка 
americ_levels = func_load('American_levels.xlsx')
display(americ_levels.head(5))

# загрузим файл, в котором хранится информация о сложности фильмов
movies_levels = func_load('movies_labels.xlsx')
display(movies_levels)

,A1_lite,A2_lite,B1_lite,B2_lite,B2_hard,C1_hard
0,a,ability,academic,abandon,absorb,acceptance
1,an,able,access,absolute,abstract,accessible
2,about,abroad,accommodation,academic,accent,accomplishment
3,above,accept,account,acceptable,accidentally,accordance
4,across,accident,achievement,accompany,accommodate,accordingly


,A1_lite,A2_lite,B1_lite,B2_lite,B2_hard,C1_hard
0,action,ability,absolutely,abandon,absorb,abolish
1,activity,able,academic,abroad,abstract,abortion
2,actor,accept,access,absolute,accent,absence
3,actress,accident,account,acceptable,accidentally,absent
4,add,according to,achievement,accompany,accommodate,absurd


,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2
...,...,...,...
223,223,Matilda(2022),C1
224,224,Bullet train,B1
225,225,Thor: love and thunder,B2
226,226,Lightyear,B2


## Формирование фрейма данных с частотами слов

In [5]:
# Соберём единый массив, который будет содержать все уникальные слова
united = pd.Series(united_words(english_levels, americ_levels)).unique()

# получим список файлов для разметки
list_files = func_list_files('Subtitles/')
# удалим расширение файлов ".srt"
list_files = [remove_symbol(file,['Subtitles/', '.srt']) for file in list_files]

In [6]:
%%time

# сформируем фрейм данных с частотами повторений слов
data = text_markup('Subtitles/', united)
# добавим в фрейм поле с названиями фильмов
data['Movie'] = list_files

# объединим фреймы "сложность фильмов" и "частоты повторений слов"
data = data.merge(movies_levels, how='left', on='Movie')

display(data.head(5))

,a,an,about,above,across,action,activity,actor,actress,add,...,rumor,setup,skeptical,sophomore,superintendent,tumor,unconstitutional,Movie,id,Level
0,0.017082,0.002749,0.003534,0.000982,0.000196,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,10_Cloverfield_lane(2016),0.0,B1
1,0.021591,0.001570,0.003664,0.000000,0.000000,0.000131,0.0,0.0,0.0,0.000000,...,0.000262,0.000131,0.0,0.000262,0.0,0.0,0.0,10_things_I_hate_about_you(1999),1.0,B1
2,0.021688,0.000744,0.001363,0.000000,0.000000,0.000124,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,Aladdin(1992),4.0,A2
3,0.017564,0.001008,0.001584,0.000000,0.000000,0.000144,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,All_dogs_go_to_heaven(1989),5.0,A2
4,0.021161,0.001719,0.004100,0.000132,0.000000,0.000000,0.0,0.0,0.0,0.000132,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,AmericanBeauty1999.BRRip,105.0,B1


Wall time: 4min 9s
